#**Nitrogen Dioxide analysis for 0.25 degree during covid-19 1st wave and 2nd wave**
The data is on daily basis. The correlation between daily NO2 levels and covidoxford data attributes is to found for all cities 

In [19]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.options.mode.use_inf_as_na = True
import numpy as np
import os
import math
from scipy.stats import pearsonr as corr
from matplotlib import pyplot as plt
#import statsmodels.api as sm
import datetime as dt
import statsmodels.formula.api as sm

In [20]:
OMINO2_PATH="omino2/data/" #NO2 data path
COVID_PATH="covidoxford/data/" #covidoxford path
CITIES_PATH="cities/data/" #cities data path

# **First Wave**
250 days from 20 march 2020

In [21]:
date=dt.date(2020,3,20)
dates=[str(int(date.day))+"."+str(int(date.month))+"."+str(int(date.year))]
for i in range(250):
    date=date+dt.timedelta(days=1)
    dates.append(str(int(date.day))+"."+str(int(date.month))+"."+str(int(date.year)))

In [32]:
cities_df=pd.read_csv(CITIES_PATH+"cities_0.25degx0.25deg.csv",error_bad_lines=False)
cities_df.head(3)

b'Skipping line 302: expected 5 fields, saw 6\n'


,location,city,population,elevation,cities
0,10.875#79.375,Kumbakonam,218835,29,Swamimalai#Kumbakonam#Koradachcheri#Valangaima...
1,28.875#77.125,Narela,1137929,222,Sonipat#Narela#Bawana#Alipur#Baghpat
2,11.125#77.875,Sivagiri,29832,165,Sivagiri#Kodumudi


In [23]:
covid_df=pd.read_csv(COVID_PATH+"covid_trends_daily.csv")
covid_df=covid_df[covid_df["date"].isin(dates)]
covid_df["metric"].unique()

array(['retail_and_recreation', 'grocery_and_pharmacy', 'residential',
       'transit_stations', 'parks', 'workplaces',
       'new_cases_smoothed_per_million',
       'new_deaths_smoothed_per_million', 'stringency_index'],
      dtype=object)

**correlation of NO2 with different attributes in covid trend daily**

In [24]:
covid_metric_df=covid_df[covid_df["metric"]=="retail_and_recreation"][["date","value"]]
covid_metric_df.head(5)

,date,value
32,20.3.2020,-13.000
33,21.3.2020,-16.286
34,22.3.2020,-25.857
35,23.3.2020,-32.429
36,24.3.2020,-40.571


In [25]:
omino2_df=pd.read_csv(OMINO2_PATH+"indian_cities_daily_0.25degx0.25deg_omino2.csv")
omino2_df.head(3)

,location,week,metric,mean
0,17.375#77.375,1.1.2019,ColumnAmountNO2,2.880367e+15
1,17.375#77.875,1.1.2019,ColumnAmountNO2,2.297635e+15
2,17.875#76.375,1.1.2019,ColumnAmountNO2,2.436043e+15


**Metric**:ColumnAmountNO2

In [26]:
metric="ColumnAmountNO2"
metric_df=omino2_df[(omino2_df["metric"]==metric)&(omino2_df["mean"]>-9999)&(~omino2_df["mean"].isna())]

In [27]:
metric_df=metric_df.merge(covid_metric_df,left_on="week",right_on="date",how="inner")

In [30]:
metric_data=metric_df.groupby(["location"])['mean'].apply(list).reset_index(name="series").set_index("location")["series"].to_dict()
covid_data=metric_df.groupby(["location"])['value'].apply(list).reset_index(name="series").set_index("location")["series"].to_dict()
result={}
for location in metric_data:
    (c,p)=corr(metric_data[location],covid_data[location])
    if p<0.05:
        result[location]=round(c,2)
result_df = pd.DataFrame(list(result.items()),columns=["location","corr"])
result_df.head(3)

,location,corr
0,10.125#76.125,0.27
1,10.125#76.375,0.47
2,10.125#76.625,0.46


In [33]:
result_df=result_df.merge(cities_df,on="location",how="right")
result_df["corr"]=result_df["corr"].fillna(0.0)
result_df["corr"].describe()

count    2244.000000
mean        0.515588
std         0.089013
min         0.170000
25%         0.460000
50%         0.530000
75%         0.580000
max         0.710000
Name: corr, dtype: float64

In [34]:
print("average correlation =",round(result_df[result_df["population"]>=1000000]["corr"].mean(),2))
result_df[result_df["population"]>=1000000]

average correlation = 0.52


,location,corr,city,population,elevation,cities
1,28.875#77.125,0.39,Narela,1137929,222,Sonipat#Narela#Bawana#Alipur#Baghpat
77,18.625#73.875,0.65,Pune,5336520,571,Dehu#Shivaji Nagar#Khadki#Alandi#Pimpri#Lohoga...
97,12.875#77.625,0.46,Bengaluru,5104047,914,Bengaluru
110,22.625#88.375,0.65,Kolkata,8539531,9,Krishnapur#Madhyamgram#Barasat#Sodpur#Khardah#...
114,28.625#77.125,0.38,Delhi,12927387,222,Shahdara#Deoli#New Delhi#Delhi#Nangloi Jat#Pit...
137,22.875#88.375,0.59,Bhatpara,2641204,13,Naihati#Shrirampur#Kesabpur#Bhatpara#Ramchandr...
145,13.125#80.375,0.47,Chennai,4663073,9,Chennai#Chinnasekkadu#George Town#Kattivakkam#...
155,8.625#77.625,0.31,Tirunelveli,1483378,82,Kalakkadu#Vadakku Viravanallur#Tirunelveli
165,25.625#85.125,0.52,Patna,1967313,57,Hajipur#Dighwara#Patna#Dinapore#Khagaul
223,16.625#80.625,0.58,Vijayawada,1016301,30,Vijayawada#Kondapalle#Amaravati


**Takeaway:**
- correlation of daily NO2 levels with daily retail_and_recreation: 0.52
- correlation of daily NO2 levels with daily residential: -0.4
- correlation of daily NO2 levels with daily transit_stations: 0.48
- correlation of daily NO2 levels with daily new_cases_smoothed_per_million: 0.5
- correlation of daily NO2 levels with daily stringency_index: -0.44

# **Second Wave**
180 days from 10 feb.

In [35]:
date=dt.date(2021,2,10)
dates=[str(int(date.day))+"."+str(int(date.month))+"."+str(int(date.year))]
for i in range(180):
    date=date+dt.timedelta(days=1)
    dates.append(str(int(date.day))+"."+str(int(date.month))+"."+str(int(date.year)))

In [44]:
cities_df=pd.read_csv(CITIES_PATH+"cities_0.25degx0.25deg.csv",error_bad_lines=False)

b'Skipping line 302: expected 5 fields, saw 6\n'


In [45]:
covid_df=pd.read_csv(COVID_PATH+"covid_trends_daily.csv")
covid_df=covid_df[covid_df["date"].isin(dates)]
covid_df["metric"].unique()

array(['retail_and_recreation', 'grocery_and_pharmacy', 'residential',
       'transit_stations', 'parks', 'workplaces',
       'new_cases_smoothed_per_million',
       'new_deaths_smoothed_per_million', 'stringency_index'],
      dtype=object)

In [46]:
covid_metric_df=covid_df[covid_df["metric"]=="stringency_index"][["date","value"]]

In [47]:
omino2_df=pd.read_csv(OMINO2_PATH+"indian_cities_daily_0.25degx0.25deg_omino2.csv")

In [48]:
metric="ColumnAmountNO2"
metric_df=omino2_df[(omino2_df["metric"]==metric)&(omino2_df["mean"]>-9999)&(~omino2_df["mean"].isna())]
metric_df.head(3)

,location,week,metric,mean
0,17.375#77.375,1.1.2019,ColumnAmountNO2,2.880367e+15
1,17.375#77.875,1.1.2019,ColumnAmountNO2,2.297635e+15
2,17.875#76.375,1.1.2019,ColumnAmountNO2,2.436043e+15


In [49]:
metric_df=metric_df.merge(covid_metric_df,left_on="week",right_on="date",how="inner")

In [50]:
metric_data=metric_df.groupby(["location"])['mean'].apply(list).reset_index(name="series").set_index("location")["series"].to_dict()
covid_data=metric_df.groupby(["location"])['value'].apply(list).reset_index(name="series").set_index("location")["series"].to_dict()
result={}
for location in metric_data:
    (c,p)=corr(metric_data[location],covid_data[location])
    if p<0.05:
        result[location]=round(c,2)
result_df = pd.DataFrame(list(result.items()),columns=["location","corr"])

In [51]:
result_df=result_df.merge(cities_df,on="location",how="right")
result_df["corr"]=result_df["corr"].fillna(0.0)
result_df["corr"].describe()

count    2244.000000
mean       -0.106756
std         0.230773
min        -0.720000
25%        -0.290000
50%         0.000000
75%         0.000000
max         0.390000
Name: corr, dtype: float64

In [52]:
print("average correlation =",round(result_df[result_df["population"]>=1000000]["corr"].mean(),2))
result_df[result_df["population"]>=1000000]

average correlation = -0.11


,location,corr,city,population,elevation,cities
1,28.875#77.125,-0.43,Narela,1137929,222,Sonipat#Narela#Bawana#Alipur#Baghpat
77,18.625#73.875,0.00,Pune,5336520,571,Dehu#Shivaji Nagar#Khadki#Alandi#Pimpri#Lohoga...
97,12.875#77.625,0.00,Bengaluru,5104047,914,Bengaluru
110,22.625#88.375,0.00,Kolkata,8539531,9,Krishnapur#Madhyamgram#Barasat#Sodpur#Khardah#...
114,28.625#77.125,-0.32,Delhi,12927387,222,Shahdara#Deoli#New Delhi#Delhi#Nangloi Jat#Pit...
137,22.875#88.375,0.16,Bhatpara,2641204,13,Naihati#Shrirampur#Kesabpur#Bhatpara#Ramchandr...
145,13.125#80.375,0.00,Chennai,4663073,9,Chennai#Chinnasekkadu#George Town#Kattivakkam#...
155,8.625#77.625,0.20,Tirunelveli,1483378,82,Kalakkadu#Vadakku Viravanallur#Tirunelveli
165,25.625#85.125,0.00,Patna,1967313,57,Hajipur#Dighwara#Patna#Dinapore#Khagaul
223,16.625#80.625,0.00,Vijayawada,1016301,30,Vijayawada#Kondapalle#Amaravati


**Takeaway:**
- correlation of daily NO2 levels with daily retail_and_recreation: 0.08
- correlation of daily NO2 levels with daily residential: -0.09
- correlation of daily NO2 levels with daily transit_stations: 0.09
- correlation of daily NO2 levels with daily new_cases_smoothed_per_million: -0.03
- correlation of daily NO2 levels with daily stringency_index: -0.1

The correlations for majority cities are 0 (uncorrelated).

**Takeaway:**
The correlations of daily NO2 levels for indian cities with covid oxford data are good for first wave and very weak for second wave